In [1]:
%load_ext sql
%sql duckdb://

# INNER JOIN, LEFT JOIN, RIGHT JOIN, FULL OUTER JOIN
Explanation:
- The code snippet demonstrates the usage of different types of joins (INNER JOIN, LEFT JOIN, RIGHT JOIN, FULL OUTER JOIN) in SQL.
- Two tables, `employees` and `departments`, are created with sample data.
- Each join type is demonstrated separately with appropriate comments.
- The expected output is mentioned after each query.
- Additional conditions are also shown to filter the results based on specific criteria.

In [2]:
%%sql

CREATE TABLE employees (
  id INT PRIMARY KEY,
  name VARCHAR(50),
  department_id INT
);

CREATE TABLE departments (
  id INT PRIMARY KEY,
  name VARCHAR(50)
);

INSERT INTO employees (id, name, department_id)
VALUES (1, 'John Doe', 1),
       (2, 'Jane Smith', 2),
       (3, 'Mike Johnson', 1),
       (4, 'Emily Brown', 3),
       (5, 'No Department', NULL),
       (6, 'No Matching', 5);

INSERT INTO departments (id, name)
VALUES (1, 'Sales'),
       (2, 'Marketing'),
       (3, 'Finance'),
       (4, 'Non-Match');

Running query in 'duckdb://'

Count


## Inner Join

All combinations that match the condition.

Non-matching combinations left out, including rows that don't participate in a match.

In [3]:
%%sql 

SELECT employees.name, departments.name
FROM employees
INNER JOIN departments ON employees.department_id = departments.id;

Running query in 'duckdb://'

name,name_1
John Doe,Sales
Jane Smith,Marketing
Mike Johnson,Sales
Emily Brown,Finance


## Left Join

All rows from left included.

In [4]:
%%sql

SELECT employees.name, departments.name
FROM employees
LEFT JOIN departments ON employees.department_id = departments.id;

Running query in 'duckdb://'

name,name_1
John Doe,Sales
Jane Smith,Marketing
Mike Johnson,Sales
Emily Brown,Finance
No Department,None
No Matching,None


## Right Join

All rows from right included.

In [5]:
%%sql

SELECT employees.name, departments.name
FROM employees
RIGHT JOIN departments ON employees.department_id = departments.id;

Running query in 'duckdb://'

name,name_1
Mike Johnson,Sales
Jane Smith,Marketing
Emily Brown,Finance
John Doe,Sales
None,Non-Match


## Full Outer Join

All rows from both tables, matched up when possible.

In [6]:
%%sql

SELECT employees.name, departments.name
FROM employees
FULL OUTER JOIN departments ON employees.department_id = departments.id;

Running query in 'duckdb://'

name,name_1
John Doe,Sales
Jane Smith,Marketing
Mike Johnson,Sales
Emily Brown,Finance
No Department,None
No Matching,None
None,Non-Match


## Left and Right Outer Join

These are synonymous with `LEFT JOIN` and `RIGHT JOINT`, and the `OUTER` keyword is optional.

## Additional Conditions

In [7]:
%%sql

SELECT employees.name, departments.name
FROM employees
INNER JOIN departments ON employees.department_id = departments.id
WHERE employees.name LIKE 'J%';

Running query in 'duckdb://'

name,name_1
John Doe,Sales
Jane Smith,Marketing


In [8]:
%%sql

SELECT employees.name, departments.name
FROM employees
LEFT JOIN departments ON employees.department_id = departments.id
WHERE departments.name IS NULL;

Running query in 'duckdb://'

name,name_1
No Department,None
No Matching,None


In [9]:
%%sql

SELECT employees.name, departments.name
FROM employees
FULL OUTER JOIN departments ON employees.department_id = departments.id
WHERE employees.name IS NULL OR departments.name IS NULL;

Running query in 'duckdb://'

name,name_1
No Department,None
No Matching,None
None,Non-Match


# CROSS JOIN, SELF JOIN
The code snippet demonstrates the usage of CROSS JOIN and SELF JOIN in SQL.

CROSS JOIN returns the Cartesian product of two tables, which means it combines each row from the first table with every row from the second table. In the example, it combines the "Employees" table with the "Departments" table, resulting in a new table with all possible combinations of employees and departments.

SELF JOIN is used to join a table with itself. In the example, it joins the "Employees" table with itself based on the condition that the EmployeeID of the first employee should not be equal to the EmployeeID of the second employee. This results in a table that shows all possible combinations of employees.

Both CROSS JOIN and SELF JOIN can be useful in scenarios where you need to generate all possible combinations or compare records within the same table.

Note: In practice, it is important to use appropriate filtering conditions to avoid generating large result sets or unnecessary combinations.
```

In [11]:
%%sql

CREATE TABLE Employees2 (
  EmployeeID INT,
  EmployeeName VARCHAR(50)
);

INSERT INTO Employees2 (EmployeeID, EmployeeName)
VALUES (1, 'John'),
       (2, 'Jane'),
       (3, 'Mike');

CREATE TABLE Departments2 (
  DepartmentID INT,
  DepartmentName VARCHAR(50)
);

INSERT INTO Departments2 (DepartmentID, DepartmentName)
VALUES (1, 'Sales'),
       (2, 'Marketing'),
       (3, 'Finance');

Running query in 'duckdb://'

Count


## Cross Join

All combinations of rows with no regard for any kind of match.

In [12]:
%sql SELECT * FROM Employees2 CROSS JOIN Departments2;

Running query in 'duckdb://'

EmployeeID,EmployeeName,DepartmentID,DepartmentName
1,John,1,Sales
2,Jane,1,Sales
3,Mike,1,Sales
1,John,2,Marketing
2,Jane,2,Marketing
3,Mike,2,Marketing
1,John,3,Finance
2,Jane,3,Finance
3,Mike,3,Finance


## Self Join

This is not a special syntax but is just something you can do by aliasing a table as two different table names in a query.

In [13]:
%%sql

SELECT e1.EmployeeName AS Employee1, e2.EmployeeName AS Employee2
FROM Employees2 e1
JOIN Employees2 e2 ON e1.EmployeeID <> e2.EmployeeID;

Running query in 'duckdb://'

Employee1,Employee2
Jane,John
Mike,John
John,Jane
Mike,Jane
John,Mike
Jane,Mike
